# 🔍 ChromaDB Viewer

Notebook per visualizzare e esplorare il contenuto del database ChromaDB.

In [11]:
import sys
from pathlib import Path

# Aggiungi source al path per importare i moduli
sys.path.insert(0, str(Path("../source").resolve()))

from AI_RM import EmbeddingStore, Embedder, default_config
print("Moduli caricati")

Moduli caricati


## 1. Connessione al Database

In [12]:
# Inizializza lo store
store = EmbeddingStore(config=default_config.vector_store)

# Mostra statistiche
stats = store.get_stats()
print(f"Database Statistics")
print(f"\tCollection: {stats.collection_name}")
print(f"\tDocuments: {stats.total_documents}")
print(f"\tPath: {stats.persist_path}")
print(f"\tMetric: {stats.distance_metric}")

Database Statistics
	Collection: krss_videos
	Documents: 0
	Path: /app/data/embeddings/chroma_db
	Metric: cosine


## 2. Visualizza Sample di Video

In [13]:
# Mostra alcuni video casuali
samples = store.sample(n=5)

if samples:
    print("Sample Videos:")
    for i, s in enumerate(samples, 1):
        print(f"\n{i}. {s['video_id']}")
        print(f"\tText: {s['text']}")
        print(f"\tCategory: {s['metadata'].get('category', 'N/A')}")
        print(f"\tChannel: {s['metadata'].get('channel_name', 'N/A')}")
else:
    print("Database vuoto! Esegui prima il notebook 02_chromadb_setup.ipynb")

Database vuoto! Esegui prima il notebook 02_chromadb_setup.ipynb


## 3. Esporta come DataFrame

In [14]:
# Esporta il DB come DataFrame per analisi
import pandas as pd

# Ottieni un DataFrame dal store (contiene id, text, metadata, ecc.)
df = store.to_dataframe()
display(df.head())
print(f'Totale video nel DB: {len(df)}')

# Esempio: crea una lista di categorie dall'indice metadata
# categories = df['metadata'].apply(lambda m: m.get('category', 'Unknown')) if 'metadata' in df.columns else None

# Nota: non reinserire dati nel DB da questo notebook; questo viewer è solo per esplorazione.

""


Totale video nel DB: 0


## 4. Distribuzione per Categoria

In [15]:
import matplotlib.pyplot as plt

categories = store.get_category_distribution()

if categories:
    plt.figure(figsize=(10, 6))
    plt.barh(list(categories.keys()), list(categories.values()))
    plt.xlabel('Numero Video')
    plt.ylabel('Categoria')
    plt.title('Distribuzione Video per Categoria')
    plt.tight_layout()
    plt.show()
    
    print("\nDettaglio:")
    for cat, count in sorted(categories.items(), key=lambda x: -x[1]):
        print(f"   {cat}: {count}")
else:
    print("Nessuna categoria trovata")

Nessuna categoria trovata


## 5. Distribuzione per Canale

In [16]:
channels = store.get_channel_distribution()

if channels:
    # Top 15 canali
    top_channels = dict(sorted(channels.items(), key=lambda x: -x[1])[:15])
    
    plt.figure(figsize=(10, 6))
    plt.barh(list(top_channels.keys()), list(top_channels.values()))
    plt.xlabel('Numero Video')
    plt.ylabel('Canale')
    plt.title('Top 15 Canali per Numero Video')
    plt.tight_layout()
    plt.show()
else:
    print("Nessun canale trovato")

Nessun canale trovato


## 6. Test Ricerca Semantica

In [17]:
# Inizializza embedder per le query (usa la config centrale)
embedder = Embedder(config=default_config.embedding)

def search(query: str, n: int = 5, category: str = None):
    """Funzione helper per cercare video."""
    query_emb = embedder.encode(query)
    where_filter = {"category": category} if category else None
    
    results = store.search(
        query_embedding=query_emb,
        n_results=n,
        where=where_filter
    )
    
    print(f"Query: '{query}'")
    if category:
        print(f"\tFilter: category = '{category}'")
    print(f"\nRisultati ({len(results)}):")
    
    for i, r in enumerate(results, 1):
        print(f"\n{i}. [{r.score:.3f}] {r.text[:70]}...")
        print(f"   ID: {r.video_id}")
        print(f"   Channel: {r.metadata.get('channel_name', 'N/A')}")
        print(f"   Category: {r.metadata.get('category', 'N/A')}")

In [18]:
# Esempio di ricerca
search("mathematic", n=5)

Query: 'mathematic'

Risultati (0):


In [19]:
# Ricerca con filtro categoria
search("machine learning", n=3, category="Education")

Query: 'machine learning'
	Filter: category = 'Education'

Risultati (0):


## 7. Lookup Video Specifico

In [20]:
# Cerca un video specifico per ID
# Sostituisci con un ID reale dal tuo database

video_id = df['video_id'].iloc[0] if not df.empty else None

if video_id:
    video = store.get_video(video_id)
    if video:
        print(f"Video: {video_id}")
        print(f"\nText: {video['text'][:200]}...")
        print(f"\nMetadata: {video['metadata']}")
        print(f"\nEmbedding shape: {video['embedding'].shape if video['embedding'] is not None else 'N/A'}")
else:
    print("Nessun video nel database")

Nessun video nel database
